In [6]:
import sys
#   paths
ANKI_PATH = 'C:/Users/plk2033/Documents/PythonDocs/anki-master'
COLLECTION_PATH = "C:/Users/plk2033/Documents/PythonDocs/collection.anki2"

sys.path.append(ANKI_PATH)
import anki
import time
from anki import Collection
from anki.utils import intTime

In [56]:
col = Collection(COLLECTION_PATH)

dID_7 = col.decks.id('Exam 7')
dID_9 = col.decks.id('Exam 9')

##Exam 7
query="select id,nid,did from cards where did = " + str(dID_7)
Exam_7_cards = col.db.all(query)

##Exam 9
query="select id,nid,did from cards where did = " + str(dID_9)
Exam_9_cards = col.db.all(query)

col.close()

In [63]:
col.findCards("cid:1482373816799")

[]

In [72]:
query="select * from revlog where cid = 1482373816799"
result = col.db.all(query)

AttributeError: 'NoneType' object has no attribute 'all'

In [25]:
col.tags.all() 

['Ch23',
 'Ch6',
 'Patrik',
 'Sahasrabuddhe',
 'Cummins',
 'Ch7',
 'Teng&Perkins',
 'marked',
 'Source',
 'ISO',
 'Coval',
 'CATBonds',
 'Capital',
 'Panning',
 'Robertson',
 'NCCI-1',
 'NCCI-2',
 'Exam9',
 'Verrall',
 'Exam',
 'Mack1994',
 'BKM',
 'Goldfarb',
 'ASOP12',
 'Bailey&Simon',
 'Bodoff',
 'RF',
 'Clark-8',
 'Exam8',
 '7',
 'RobbinUW',
 'CH9',
 'Hurlimann',
 'CH11',
 'Shapland',
 'Ch12',
 'Ch10',
 'Ch11',
 'Ch16',
 'Ch14',
 'Ch15',
 'Couret&Venter',
 'Fisher',
 'Ch1',
 'BrehmCh5',
 'Ch3',
 'BrehmCh3',
 'BrehmCh2',
 'BrehmCh1',
 'Feldblum',
 'Ch8',
 'Ch9',
 'Kreps',
 'Meyers',
 'Marshall',
 'GLM',
 'Brehm',
 'Taylor',
 'Brosius',
 'Mack2000',
 'Clark',
 'Exam7',
 'McClenahan',
 'Venter',
 'Mahler1',
 'Ch2',
 'Mango',
 'Grossi',
 'Bahnemann',
 'BrehmCh4',
 'Questions',
 'RobbinIRR',
 'Ferrari',
 'Butsic',
 'Siewert',
 'Bernegger']